In [1]:
import pandas as pd
import numpy as np
import scipy
import scipy.stats as sp
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
A = np.array([12,20,39,17,23,32,21,23,10,9,17,24])
B = np.array([21,17,14,40,31,29,34,18,14,25,9,28])

In [3]:
AB = np.array([np.append(np.zeros(len(A)), np.ones(len(B))), np.append(A,B)])
AB


array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [12., 20., 39., 17., 23., 32., 21., 23., 10.,  9., 17., 24., 21.,
        17., 14., 40., 31., 29., 34., 18., 14., 25.,  9., 28.]])

In [4]:
AB_t = AB.T
AB_t

array([[ 0., 12.],
       [ 0., 20.],
       [ 0., 39.],
       [ 0., 17.],
       [ 0., 23.],
       [ 0., 32.],
       [ 0., 21.],
       [ 0., 23.],
       [ 0., 10.],
       [ 0.,  9.],
       [ 0., 17.],
       [ 0., 24.],
       [ 1., 21.],
       [ 1., 17.],
       [ 1., 14.],
       [ 1., 40.],
       [ 1., 31.],
       [ 1., 29.],
       [ 1., 34.],
       [ 1., 18.],
       [ 1., 14.],
       [ 1., 25.],
       [ 1.,  9.],
       [ 1., 28.]])

A shuffled data with labels 
0 -> A
1 -> B

In [5]:
np.random.shuffle(AB_t)
AB_t

array([[ 1., 18.],
       [ 1., 17.],
       [ 1., 25.],
       [ 1., 14.],
       [ 0., 32.],
       [ 0., 17.],
       [ 1.,  9.],
       [ 0.,  9.],
       [ 0., 12.],
       [ 1., 29.],
       [ 1., 28.],
       [ 1., 14.],
       [ 0., 23.],
       [ 1., 21.],
       [ 0., 17.],
       [ 0., 20.],
       [ 0., 10.],
       [ 1., 40.],
       [ 0., 21.],
       [ 0., 39.],
       [ 1., 31.],
       [ 1., 34.],
       [ 0., 23.],
       [ 0., 24.]])

Create random lebels for the datas -> Chance

In [6]:
experimental_data = np.array([np.random.randint(0,2,len(AB_t)), AB_t[:, 1]]).T
experimental_data 

array([[ 1., 18.],
       [ 1., 17.],
       [ 1., 25.],
       [ 1., 14.],
       [ 1., 32.],
       [ 0., 17.],
       [ 1.,  9.],
       [ 1.,  9.],
       [ 1., 12.],
       [ 1., 29.],
       [ 0., 28.],
       [ 1., 14.],
       [ 1., 23.],
       [ 0., 21.],
       [ 0., 17.],
       [ 0., 20.],
       [ 0., 10.],
       [ 1., 40.],
       [ 1., 21.],
       [ 0., 39.],
       [ 1., 31.],
       [ 0., 34.],
       [ 1., 23.],
       [ 1., 24.]])

In [ ]:
def ab_test(group_a:list, group_b:list, count:int):
    mean_diffs = np.empty([count,1])
    len_a = len(group_a)
    len_b = len(group_b)
    combined_group = np.append(group_a,group_b)
    label_a = np.ones(len_a)
    label_b = np.zeros(len_b)
    combined_label = np.append(label_a, label_b)
    combined_data = np.column_stack(combined_label, combined_group)
    np.random.shuffle(combined_data)
    len_combined_data = len(combined_data)
    for i in range(count):
        experiment_label = np.random.randint(0, 2, len_combined_data)
        experiment_data = np.array([np.append(experiment_label,combined_data_t[:, 1])])
        mean_diffs[i] = experiment_data[experiment_data[:0] == 1][:, 1].mean() - experiment_data[experiment_data[:0] == 0][:, 1].mean()
    return mean_diffs

In [32]:
np.column_stack((A,B))

array([[12, 21],
       [20, 17],
       [39, 14],
       [17, 40],
       [23, 31],
       [32, 29],
       [21, 34],
       [23, 18],
       [10, 14],
       [ 9, 25],
       [17,  9],
       [24, 28]])